In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
PATH_OF_DATA = '/content/gdrive/MyDrive/images'

In [ ]:
import shutil
import os
import math
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt

In [ ]:
os.chdir(PATH_OF_DATA)
category_list = list(filter(lambda x: os.path.isdir(x), os.listdir()))
for category in category_list:
  print(category)

Chilli
Bitter gourd
Lime
Mature tea tree
White Gigantic swallow wort
Ground nut
Wood-apple
Banana pepper


In [ ]:
data_set_dirs= ['train','test']
for dsdirs in data_set_dirs:
  path = PATH_OF_DATA + '/'+ dsdirs
  os.mkdir( path,755 )

In [ ]:
train_prop = 0.7

In [ ]:
def create_dataset():
  for ii,cat in enumerate(category_list):    
    src_path = PATH_OF_DATA + '/' + cat
    dest_dir1 = PATH_OF_DATA+'/train/'+str(ii)
    dest_dir2 = PATH_OF_DATA+'/test/'+str(ii)
    
    dest_dirs_list = [dest_dir1,dest_dir2]
    for dirs in dest_dirs_list:
      os.mkdir(dirs,755 )
    
    #get files' names list from respective directories
    os.chdir(src_path)
    files = [f for f in os.listdir() if os.path.isfile(f)]
    
    #get training, testing  files count
    train_count = math.ceil(train_prop*len(files))
    test_count = int((len(files)-train_count)/2)
    
    #get files to segragate for train,test data set
    train_data_list = files[0: train_count] 
    test_data_list =  files[train_count+test_count:]
       
  
    for train_data in train_data_list:
      train_path = src_path + '/' + train_data
      shutil.copy(train_path,dest_dir1)
    
    for test_data in test_data_list:
      test_path = src_path + '/' + test_data
      shutil.copy(test_path,dest_dir2)    
    
create_dataset()

In [ ]:
train_dir = '/content/gdrive/MyDrive/images/train'

test_dir = '/content/gdrive/MyDrive/images/test'

In [ ]:
import numpy 


train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, 
                                   shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, fill_mode="nearest")

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 32, target_size = (224, 224), color_mode='rgb', 
                                                    classes=None, class_mode='categorical', shuffle=True, seed=None, interpolation="nearest")

test_generator = test_datagen.flow_from_directory( test_dir, batch_size = 32, target_size = (224, 224))

test_steps_per_epoch = numpy.math.ceil(test_generator.samples / test_generator.batch_size)


Found 211 images belonging to 8 classes.
Found 43 images belonging to 8 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")

In [ ]:
base_model.load_weights("/content/gdrive/My Drive/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
num_classes = 8

In [ ]:
def recall(y_true, y_pred):
     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
     recall = true_positives / (possible_positives + K.epsilon())
     return recall

In [ ]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [ ]:
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten(name="flatten")(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.SGD(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc', recall, precision])

In [ ]:
resnet_history = model.fit(train_generator, validation_data = test_generator, epochs = 25, 
                           steps_per_epoch=train_generator.samples/train_generator.batch_size,
                           validation_steps=test_generator.samples/test_generator.batch_size)

Epoch 1/25
6/6 [==============================] - 102s 15s/step - loss: 0.6951 - acc: 0.0966 - recall: 0.0108 - precision: 0.0646 - val_loss: 0.5899 - val_acc: 0.1163 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/25
6/6 [==============================] - 90s 14s/step - loss: 0.6610 - acc: 0.1368 - recall: 0.0150 - precision: 0.1092 - val_loss: 0.5827 - val_acc: 0.1163 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 3/25
6/6 [==============================] - 91s 14s/step - loss: 0.6615 - acc: 0.1416 - recall: 0.0237 - precision: 0.2227 - val_loss: 0.5756 - val_acc: 0.1163 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 4/25
6/6 [==============================] - 91s 14s/step - loss: 0.6490 - acc: 0.1289 - recall: 0.0120 - precision: 0.0846 - val_loss: 0.5689 - val_acc: 0.1163 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 5/25
6/6 [==============================] - 89s 13s/step - loss: 0.6335 - acc: 0.1493 - recall: 0.0248 - precision: 0

In [100]:
model.evaluate_generator(test_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.470302939414978, 0.11627907305955887, 0.0, 0.0]

In [101]:
predictions = model.predict_generator(test_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = numpy.argmax(predictions, axis=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [102]:
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

In [103]:
import sklearn.metrics as metrics


report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report) 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00         7
           2       0.12      1.00      0.21         5
           3       0.00      0.00      0.00         5
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         5
           6       0.00      0.00      0.00         7
           7       0.00      0.00      0.00         6

    accuracy                           0.12        43
   macro avg       0.01      0.12      0.03        43
weighted avg       0.01      0.12      0.02        43



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [104]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________